In [1]:
import asyncio
import sys
from pathlib import Path
from dotenv import load_dotenv
import os

project_root = Path.cwd().parents[0] # ajusta si tu notebook está más adentro
sys.path.append(str(project_root))
load_dotenv()



True

In [22]:
# Cargar datos y crear dataset
from data_processing.data_loader import DataLoader
from data_processing.incident_consolidator import IncidentConsolidator
from pathlib import Path

execution_date = "2025-09-08"

# Configurar rutas explícitamente para usar 'datos' en lugar de 'datos_ejemplo'
# Desde notebooks/, subir un nivel para llegar a la raíz del proyecto
project_root = Path().resolve().parent
datos_path = project_root / "datos"

loader = DataLoader(
    base_path=datos_path,
    cv_path=datos_path / "cv",
    daily_path=datos_path / "daily_files",
    feedback_path=datos_path / "feedback"
)

source_ids = loader.get_all_source_ids()

print(f"Fecha de ejecución: {execution_date}")
print(f"Usando rutas:")
print(f"- Base: {loader.base_path}")
print(f"- CV: {loader.cv_path}")
print(f"- Daily: {loader.daily_path}")
print(f"- Feedback: {loader.feedback_path}")
print(f"Fuentes encontradas: {len(source_ids)}")
print(f"Source IDs: {source_ids}")


Fecha de ejecución: 2025-09-08
Usando rutas:
- Base: C:\Users\EQUIPO\Documents\AI\Prueba tecnica agentes\agent_factory\datos
- CV: C:\Users\EQUIPO\Documents\AI\Prueba tecnica agentes\agent_factory\datos\cv
- Daily: C:\Users\EQUIPO\Documents\AI\Prueba tecnica agentes\agent_factory\datos\daily_files
- Feedback: C:\Users\EQUIPO\Documents\AI\Prueba tecnica agentes\agent_factory\datos\feedback
Fuentes encontradas: 18
Source IDs: ['195385', '195436', '195439', '196125', '199944', '207936', '207938', '209773', '211544', '220504', '220505', '220506', '224602', '224603', '228036', '228038', '239611', '239613']


In [23]:
# Crear el dataset con todos los incidentes detectados
consolidator = IncidentConsolidator(execution_date)
dataset = consolidator.build_dataset(source_ids, loader)

print(f"Dataset creado con {len(dataset)} fuentes")
print(f"Fuentes en dataset: {list(dataset.keys())}")

# Mostrar ejemplo de una fuente
sample_source = list(dataset.keys())[0] if dataset else None
if sample_source:
    print(f"\nEjemplo de fuente {sample_source}:")
    sample_data = dataset[sample_source]
    print(f"- CV disponible: {'cv_text' in sample_data}")
    print(f"- Archivos del día: {len(sample_data.get('daily_files', []))}")
    print(f"- Archivos semana pasada: {len(sample_data.get('last_week_files', []))}")
    print(f"- Incidentes detectados: {list(sample_data.get('incidents', {}).keys())}")


DEBUG _detect_volume_variation: current=2, last_week=2
DEBUG threshold: None
DEBUG day_key: mon
DEBUG day_info: {}
DEBUG: Iniciando detección con last_week_rows=2
DEBUG _detect_volume_variation: current=0, last_week=0
DEBUG threshold: 150000
DEBUG day_key: mon
DEBUG day_info: {}
DEBUG: Iniciando detección con last_week_rows=0
DEBUG _detect_volume_variation: current=347476, last_week=313425
DEBUG threshold: None
DEBUG day_key: mon
DEBUG day_info: {}
DEBUG: Iniciando detección con last_week_rows=313425
DEBUG _detect_volume_variation: current=0, last_week=0
DEBUG threshold: 310084
DEBUG day_key: mon
DEBUG day_info: {}
DEBUG: Iniciando detección con last_week_rows=0
DEBUG _detect_volume_variation: current=179070, last_week=158655
DEBUG threshold: 506743
DEBUG day_key: mon
DEBUG day_info: {}
DEBUG: Iniciando detección con last_week_rows=158655
DEBUG _detect_volume_variation: current=4060, last_week=4346
DEBUG threshold: 37346
DEBUG day_key: mon
DEBUG day_info: {}
DEBUG: Iniciando detección 

In [24]:
# Crear las herramientas para el agente
from report_builder import build_incident_toolkit

tools = build_incident_toolkit(dataset, execution_date)
print(f"Herramientas creadas: {len(tools)}")
print(f"Nombres de herramientas: {[tool.__name__ for tool in tools]}")


Herramientas creadas: 3
Nombres de herramientas: ['list_sources', 'get_source_cv_and_data', 'get_execution_date_info']


In [25]:
# Probar las herramientas limpias
print("=== PROBANDO HERRAMIENTAS LIMPIAS ===")

# 1. list_sources
list_sources = next(tool for tool in tools if tool.__name__ == "list_sources")
print("\n1. list_sources():")
sources_result = list_sources()
print(sources_result)

# 2. get_source_cv_and_data para la fuente 195385
get_source_cv_and_data = next(tool for tool in tools if tool.__name__ == "get_source_cv_and_data")
print(f"\n2. get_source_cv_and_data('195385'):")
cv_result = get_source_cv_and_data('195385')
print(cv_result[:1000] + "..." if len(cv_result) > 1000 else cv_result)

print(f"\n✅ Toolkit limpio con {len(tools)} herramientas:")
for tool in tools:
    print(f"  - {tool.__name__}")


=== PROBANDO HERRAMIENTAS LIMPIAS ===

1. list_sources():
{"execution_date": "2025-09-08", "sources": [{"source_id": "195385", "display_name": "_Settlement_Layout_2", "files_today": 1, "files_last_weekday": 1, "expected_files": 0, "missing_estimate": null, "has_cv": true, "first_upload_utc": "2025-09-08T08:06:47.089856+00:00", "last_upload_utc": "2025-09-08T08:06:47.089856+00:00"}, {"source_id": "195436", "display_name": "MyPal_DBR RX", "files_today": 1, "files_last_weekday": 1, "expected_files": 1, "missing_estimate": 0, "has_cv": true, "first_upload_utc": "2025-09-08T15:10:39.680029+00:00", "last_upload_utc": "2025-09-08T15:10:39.680029+00:00"}, {"source_id": "195439", "display_name": "MyPal_Activity report", "files_today": 1, "files_last_weekday": 1, "expected_files": 1, "missing_estimate": 0, "has_cv": true, "first_upload_utc": "2025-09-08T02:34:02.058668+00:00", "last_upload_utc": "2025-09-08T02:34:02.058668+00:00"}, {"source_id": "196125", "display_name": "_Settlement_Layout_1", 

In [26]:
# Configurar y ejecutar el agente
import asyncio
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from adk_components.agent_definition import create_report_agent
from config import settings

async def run_agent_with_prompt(prompt: str):
    """Ejecuta el agente con un prompt específico"""
    agent = create_report_agent(tools)
    session_service = InMemorySessionService()
    session = await session_service.create_session(
        app_name=settings.APP_NAME, user_id=settings.USER_ID
    )
    runner = Runner(agent=agent, app_name=settings.APP_NAME, session_service=session_service)
    content = types.Content(role='user', parts=[types.Part(text=prompt)])
    
    async for event in runner.run_async(user_id=session.user_id, session_id=session.id, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
            return ''
    return ''

print("Función run_agent_with_prompt definida y lista para usar")


Función run_agent_with_prompt definida y lista para usar


In [ ]:
# PRUEBA DEL NUEVO ENFOQUE "LLM + CVs"
# El agente ahora usa get_source_cv_and_data() para análisis experto


prompt_cv_analysis = f"""
OBJETIVO: Generar un reporte preciso basado únicamente en el análisis de los CVs y los datos reales.

HERRAMIENTAS DISPONIBLES (USAR SOLO ESTAS):
- list_sources(): panorama general de todas las fuentes
- get_source_cv_and_data(source_id): CV completo y datos crudos para análisis experto

INSTRUCCIONES ESPECÍFICAS:
1. USA SOLO list_sources() y get_source_cv_and_data() - NO uses otras herramientas
2. Para CADA fuente en list_sources(), usa get_source_cv_and_data() para análisis completo
3. Lee completamente el CV de cada fuente para entender sus patrones normales
4. Interpreta inteligentemente si los eventos son normales según el CV o verdaderos incidentes
5. Determina qué día de la semana es {execution_date} y verifica patrones específicos para ese día en cada CV

CASO ESPECIAL A VALIDAR:
- Fuente 195385: ¿Los archivos que llegaron son normales según su CV?
- ¿El timing 08:06 UTC está dentro de las ventanas esperadas?
- ¿El lag -1 (archivos del sábado llegando domingo) es normal según el CV?

REGLAS CRÍTICAS PARA ANÁLISIS:
- Si el CV dice que algo es normal, NO es un incidente
- Solo reporta verdaderos desvíos de los patrones del CV
- Usa números reales de records en "All Good"
- Cada fuente aparece solo una vez en la sección de mayor severidad
- IGNORA datos de "raw_incidents" si contradicen el análisis del CV

⚠️ REGLA CRÍTICA SOBRE VOLUME VARIATIONS:
- Si volume decrease es causado por missing files → NO reportar volume variation
- Solo reportar volume variation si los archivos llegaron pero con menos/más rows
- Ejemplo: Si 0 files llegaron y 0 rows → Solo missing files (NO volume variation)
- Ejemplo: Si 2 files llegaron pero con 50% menos rows → Volume variation

CLASIFICACIÓN DE SEVERIDAD:
🔴 URGENT: Missing files críticos según CV O 3+ incidentes "needs attention"
🟡 NEEDS ATTENTION: Desvíos de volumen, timing fuera de ventanas del CV
🟢 ALL GOOD: Todo dentro de patrones normales del CV

GENERATE THE EXECUTIVE REPORT IN ENGLISH for {execution_date}
"""

print("🚀 EJECUTANDO NUEVO ENFOQUE 'LLM + CVs'...")
print("="*80)

response = await run_agent_with_prompt(prompt_cv_analysis)



🚀 EJECUTANDO NUEVO ENFOQUE 'LLM + CVs'...
🤖 Usando OpenAI: openai/gpt-4.1 (temperature=0.1)


In [30]:
print(response)


*Report generated at UTC HOUR*: 16:00 UTC

*  Urgent Action Required*
• * __Payments_Layout_1_V3 (id: 220504)* – 2025-09-08: 12 files missing past 08:08–08:18 UTC — entities: Clien_CBK, WhiteLabel, Shop, Google, POC, Market, Innovation, Donation, Beneficios, ApplePay, Anota-ai, AddCard → *Action:* Notify provider to generate/re-send; re-run ingestion and verify completeness
• * _Settlement_Layout_1 (id: 196125)* – 2025-09-08: 2 files missing past 08:00–08:00 UTC — expected: 2 files (Mon | 2 | 0 | 0) → *Action:* Notify provider to generate/re-send; re-run ingestion and verify completeness
• * __Payments_Layout_2_V3 (id: 220505)* – 2025-09-08: 2 files missing past 08:00–08:00 UTC — expected: 2 files (Mon | 2 | 2 | 2) → *Action:* Notify provider to generate/re-send; re-run ingestion and verify completeness
• * __Payments_Layout_3_V3 (id: 220506)* – 2025-09-08: 1 file missing past 08:00–08:00 UTC — expected: 1 file (Mon | 1 | 1 | 1) → *Action:* Notify provider to generate/re-send; re-run i